# Espacios de color

En este notebook aprenderás a segmentar objetos utilizando el espacio de color HSV y la librería OpenCV.

Componentes HSV :
* Matiz (Hue),  es el color elegido codificado como un ángulo de 0 a 360
* Saturación, intensidad cromática, es decir, claridad-oscuridad del color (centro círculo al borde 0-100)
* Luminosidad (Value), todas las versiones de brillantez del color elegido (down 0 – top 100)

## Librerías

Para la mayor parte del trabajo que realizaremos sobre las imágenes usaremos la librería *OpenCV*, *Numpy* y *Plotly*.

### OpenCV

* Librería open source
* Proporciona las herramientas necesarias para resolver problemas de visión artificial mediante una combinación de funciones de procesamiento de imágenes de bajo nivel (detección de bordes a nivel de píxeles, umbralización) y algoritmos de alto nivel (detección de contornos y reconocimiento de formas)
* Detección de rostros, detección de peatones, entre otros
* Soporte para Java, C++, Javascript

In [1]:
import cv2

### Numpy

* Soporte para de matrices multidimensionales

In [2]:
import numpy as np

### Plotly

* Publicación de gráficos interactivos

In [3]:
import plotly.express as px

## Lectura de imágenes y el espacio de color RGB

Ejecutamos el siguiente comando para descargar una imágen.


In [4]:
!wget https://ivan-sipiran.com/downloads/Imagenes.zip
!unzip Imagenes.zip

--2025-08-08 18:34:04--  https://ivan-sipiran.com/downloads/Imagenes.zip
Resolving ivan-sipiran.com (ivan-sipiran.com)... 66.96.149.31
Connecting to ivan-sipiran.com (ivan-sipiran.com)|66.96.149.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30654084 (29M) [application/zip]
Saving to: ‘Imagenes.zip’

Imagenes.zip        100%[===================>]  29.23M  50.4MB/s    in 0.6s    

2025-08-08 18:34:05 (50.4 MB/s) - ‘Imagenes.zip’ saved [30654084/30654084]

Archive:  Imagenes.zip
   creating: Imagenes/
  inflating: Imagenes/20191123_093200.jpg  
  inflating: Imagenes/Aviones.jpg    
  inflating: Imagenes/bird.png       
  inflating: Imagenes/cameraman.tif  
  inflating: Imagenes/centro1.jpg    
  inflating: Imagenes/centro2.png    
  inflating: Imagenes/claro.png      
  inflating: Imagenes/contrast1.jpg  
  inflating: Imagenes/contrast2.jpeg  
  inflating: Imagenes/contrast3.jpg  
  inflating: Imagenes/diagonalbars.png  
  inflating: Imagenes/digits.png     

Leemos una imagen y la mostramos.

In [5]:
# Leer
img = cv2.imread('Imagenes/centro1.jpg')

# Mostrar
fig = px.imshow(img)
fig.show()

La imagen no se muestra como esperamos. Esto se debe a que *OpenCV* lee las imágenes a color en el orden BGR, es decir primero el canal azul, luego el canal  verde y finalmente el canal rojo. Cuando enviamos esta imagen a dibujar, *Plotly* cree que la imagen está en formato RGB. Para solucionar esto podemos hacer un cambio en los canales de color.

In [6]:
# Conversión a RGB
#img = img[:, :, [2, 1, 0]] # Forma 1
img = img[:,:,::-1] # Forma 2
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Forma 3

fig = px.imshow(img)
fig.show()

In [7]:
# Dimensiones?
print(img.shape)

(366, 550, 3)


In [8]:
# Color RGB en el pixel y=50,x=50?
print(img[50,50,:])

[183 143  56]


Por defecto las imágenes están en el espacio de color RGB. Sin embargo, al momento de procesarlas, otros espacios de color pueden ser más útiles. Por ejemplo, muchas de las técnicas clásicas de visión computacional trabajan sobre imágenes en escalas de grises.

In [9]:
# Convertir imagen a escala de grises
imgGray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Alternativamente, uno puede leer la imagen como escala de grises así:
# imgGray = cv2.imread('centro1.jpg',0)

fig = px.imshow(imgGray, binary_string=True)
fig.show()

In [10]:
# Dimensiones?
print(imgGray.shape)

(366, 550)


**¿Cómo segmentar la imagen?**

## Espacio de color HSV

Ahora utilizaremos el espacio de color HSV para seleccionar los pixeles de la imagen que tienen un color específico. Primero convertimos la imagen a HSV y calculamos cuál es el color HSV que queremos seleccionar.

In [11]:
# Creamos un pixel de color y lo convertimos a HSV
amarillo = np.uint8([[[255,255,0]]])

amarilloHSV = cv2.cvtColor(amarillo, cv2.COLOR_RGB2HSV)
print(amarilloHSV)

[[[ 30 255 255]]]


In [12]:
# el color rojo en HSV?
rojo = np.uint8([[[255,0,0]]])

rojoHSV = cv2.cvtColor(rojo, cv2.COLOR_RGB2HSV)
print(rojoHSV)

[[[  0 255 255]]]


In [13]:
# Creamos una paleta de colores HSV
paleta = np.zeros((180, 50, 3), np.uint8) # Filas, columnas, canales

for i in range(0, 180):
  for j in range(0, 50):
    paleta[i,j,0] = i # H ...
    paleta[i,j,1] = 128 # S
    paleta[i,j,2] = 128 # V

paletaRGB = cv2.cvtColor(paleta, cv2.COLOR_HSV2RGB)

fig = px.imshow(paletaRGB)
fig.show()

In [14]:
# Paleta para un color HSV
paleta = np.zeros((255,255,3), np.uint8)

for i in range(255):
  for j in range(255):
    paleta[i,j,0] = 60
    paleta[i,j,1] = i
    paleta[i,j,2] = j

paletaRGB = cv2.cvtColor(paleta, cv2.COLOR_HSV2RGB)

fig = px.imshow(paletaRGB)
fig.show()

Como vemos, hemos seleccionado el color amarilo (en RGB) y lo hemos convertido a HSV. Recuerda que el primer componente es la cromaticidad (30), el segundo componente es la saturación (255) y el tercer componente es la luminosidad (255).

Si nosotros queremos seleccionar el color amarillo en la imagen, necesitamos definir el mínimo y máximo amarillo en HSV. Esto es fácil solo basta con variar primero el Hue y definir los valores correctos de saturación para permitir amarillos más claros o oscuros.

In [15]:
# Rango de colores
lower_amarillo = np.array([20, 50, 128])
upper_amarillo = np.array([40, 255, 255])

# Convertimos la imagen de RGB a HSV
imgHSV = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

# Buscamos todos los pixeles que estan entre el minimo y maximo amarillo
mask = cv2.inRange(imgHSV, lower_amarillo, upper_amarillo)

# Obtenemos solo los pixeles que tienen las propiedades definidas
res = cv2.bitwise_and(img, img, mask = mask)

# Mostramos las imágenes
from plotly.subplots import make_subplots

fig = make_subplots(rows=1,cols=2)
fig.add_trace(px.imshow(mask,binary_string=True).data[0], 1, 1)
fig.add_trace(px.imshow(res).data[0], 1, 2)

fig.show()

## Ejercicio: Segmentación de células

En análisis de imágenes médicas, las imágenes de células sanguíneas en tejidos del estómago revelan la presencia de leucocitos atrofiados. Esta presencia indica una alta probabilidad de padecer de cáncer. Las células atrofiadas presentan un color azulado profundo que se diferencia del resto de células de la imagen. Utilizamos el análisis de espacio de colores para segmentar las células atrofiadas.

In [21]:
img1 = cv2.imread('Imagenes/pathology_cll20x01.jpg')
img2 = cv2.imread('Imagenes/pathology_cll40x03.jpg')

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

fig = px.imshow(img1)
fig.show()

fig = px.imshow(img2)
fig.show()

In [33]:
# Rango de colores
lower_azul = np.array([110,100,50])
upper_azul = np.array([130,255,255])

# Convertimos la imagen de RGB a HSV
imgHSV2 = cv2.cvtColor(img1, cv2.COLOR_RGB2HSV)

# Buscamos todos los pixeles que estan entre el minimo y maximo azul
mask2 = cv2.inRange(imgHSV2, lower_azul, upper_azul)

# Obtenemos solo los pixeles que tienen las propiedades definidas
res2 = cv2.bitwise_and(img2, img2, mask=mask2)

# Cambiar el fondo negro por blanco
res2_blanco = res2.copy()
res2_blanco[np.where(mask2 == 0)] = [255, 255, 255]  # blanco

# Mostramos las imágenes
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(px.imshow(mask2, binary_string=True).data[0], 1, 1)
fig.add_trace(px.imshow(res2_blanco).data[0], 1, 2)

fig.show()

## Propuesto

Realizar la segmentación de piel humana utilizando el espacio de color HSV



In [34]:
!wget -O billete.jpg "https://www.cambiator.es/wp-content/uploads/Billete-de-20.000-pesos-chilenos-reverso.jpg"

--2025-08-08 19:09:52--  https://www.cambiator.es/wp-content/uploads/Billete-de-20.000-pesos-chilenos-reverso.jpg
Resolving www.cambiator.es (www.cambiator.es)... 104.21.48.1, 104.21.16.1, 104.21.96.1, ...
Connecting to www.cambiator.es (www.cambiator.es)|104.21.48.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102884 (100K) [image/jpeg]
Saving to: ‘billete.jpg’

billete.jpg         100%[===================>] 100.47K  --.-KB/s    in 0.005s  

2025-08-08 19:09:52 (20.1 MB/s) - ‘billete.jpg’ saved [102884/102884]



In [35]:
# Leer
img = cv2.imread('billete.jpg')

# Mostrar
fig = px.imshow(img)
fig.show()